In [ ]:
!pip install uv # uv 설치

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 85.0 MB/s eta 0:00:00:00:0100:01


In [4]:
!uv pip install PublicDataReader --upgrade

Using Python 3.12.12 environment at: /usr
Resolved 16 packages in 331ms                                        
Prepared 8 packages in 644ms                                             
Uninstalled 6 packages in 93ms
Installed 8 packages in 54ms                                
 - beautifulsoup4==4.13.5
 + beautifulsoup4==4.13.3
 - numpy==2.0.2
 + numpy==2.4.0
 - pandas==2.2.2
 + pandas==2.2.3
 + publicdatareader==1.1.1.post2
 - requests==2.32.4
 + requests==2.32.3
 - soupsieve==2.8
 + soupsieve==2.8.1
 - urllib3==2.5.0
 + urllib3==2.6.2
 + xmltodict==0.14.2


In [5]:
%reset -f # 변수 초기화

Don't know how to reset  #, please run `%reset?` for details
Don't know how to reset  변수, please run `%reset?` for details
Don't know how to reset  초기화, please run `%reset?` for details


In [6]:
import PublicDataReader as pdr

In [7]:
## 시군구 Code
sigungu_code = pdr.code_bdong()
sigungu_code.head()

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2024.8.1. 시행)


,시도코드,시도명,시군구코드,시군구명,법정동코드,읍면동명,동리명,생성일자,말소일자
0,11,서울특별시,11000,,1100000000,,,19880423,
1,11,서울특별시,11110,종로구,1111000000,,,19880423,
2,11,서울특별시,11110,종로구,1111010100,청운동,,19880423,
3,11,서울특별시,11110,종로구,1111010200,신교동,,19880423,
4,11,서울특별시,11110,종로구,1111010300,궁정동,,19880423,


In [ ]:
# sigungu_code['시도명'].unique()

array(['서울특별시', '부산직할시', '대구직할시', '인천직할시', '광주직할시', '대전직할시', '부산광역시',
       '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', '세종특별자치시', '경기도',
       '북부출장소', '강원도', '동해출장소', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주도', '제주특별자치도', '강원특별자치도', '전북특별자치도'], dtype=object)

In [41]:
"""
# 제외할 시도명 목록
exclude = ['부산직할시', '대구직할시', '인천직할시', '광주직할시', '대전직할시', '북부출장소', '강원도', '동해출장소', '제주도', '전라북도']

# 컬럼 존재 확인 후 필터링
if '시도명' not in sigungu_code.columns:
    raise KeyError("sigungu_code에 '시도명' 컬럼이 없습니다.")

filtered = sigungu_code[~sigungu_code['시도명'].isin(exclude)].reset_index(drop=True)
filtered
"""

,시도코드,시도명,시군구코드,시군구명,법정동코드,읍면동명,동리명,생성일자,말소일자
0,11,서울특별시,11000,,1100000000,,,19880423,
1,11,서울특별시,11110,종로구,1111000000,,,19880423,
2,11,서울특별시,11110,종로구,1111010100,청운동,,19880423,
3,11,서울특별시,11110,종로구,1111010200,신교동,,19880423,
4,11,서울특별시,11110,종로구,1111010300,궁정동,,19880423,
...,...,...,...,...,...,...,...,...,...
41804,52,전북특별자치도,52800,부안군,5280042024,위도면,대리,20240118,
41805,52,전북특별자치도,52800,부안군,5280042025,위도면,거륜리,20240118,
41806,52,전북특별자치도,52800,부안군,5280042026,위도면,식도리,20240118,
41807,52,전북특별자치도,52800,부안군,5280042027,위도면,상왕등리,20240118,


In [47]:
# filtered['시도코드'].unique()

array(['11', '26', '27', '28', '29', '30', '31', '36', '41', '43', '44',
       '46', '47', '48', '50', '51', '52'], dtype=object)

In [ ]:
# sido_code = ['11', '26', '27', '28', '29', '30', '31', '36', '41', '43', '44', '46', '47', '48', '50', '51', '52']
# 11 서울, 26 부산, 27 대구, 28 인천, 29 광주, 30 대전, 31 울산, 36 세종, 41 경기, 43 충북, 44 충남, 46 전남, 47 경북, 48 경남, 50 제주, 51 강원, 52 전북

In [ ]:
"""
from PublicDataReader import Kbland
api = Kbland()
params = {
    "매물종별구분": "01",
    "지역코드": "26",
}
rent_ratio = api.get_jeonse_price_ratio(**params)
rent_ratio
"""

### 최종 코드

In [10]:
from PublicDataReader import Kbland
import pandas as pd

api = Kbland()
sido_code = ['11', '26', '27', '28', '29', '30', '31', '36', '41', '43', '44', '46', '47', '48', '50', '51', '52']

dfs = []
for code in sido_code:
    params = {
        "매물종별구분": "01",
        "지역코드": code,
    }
    try:
        data = api.get_jeonse_price_ratio(**params)
        df = pd.DataFrame(data)
        df['지역코드'] = code  # 혹시 누락된 경우 명시적으로 추가
        dfs.append(df)
    except Exception as e:
        print(f"지역코드 {code} 오류: {e}")

if not dfs:
    raise RuntimeError("API에서 데이터를 얻지 못했습니다.")

result_table = pd.concat(dfs, ignore_index=True)

# '전세가격비율' 컬럼명을 '전세가율'로 변경
if '전세가격비율' in result_table.columns:
    result_table.rename(columns={'전세가격비율': '전세가율'}, inplace=True)

# '전세가율' 값이 퍼센트 문자열이면 숫자로 변환 및 소수점 둘째자리로 반올림
if '전세가율' in result_table.columns:
    result_table['전세가율'] = (
        result_table['전세가율']
        .astype(str)
        .str.replace('%', '', regex=False)
        .str.replace(',', '', regex=False)
    )
    result_table['전세가율'] = pd.to_numeric(result_table['전세가율'], errors='coerce').round(2)

# '전세가율' 컬럼에서 Null 값이 있는 행 제외
result_table = result_table[result_table['전세가율'].notna()].reset_index(drop=True)

# '날짜' 컬럼을 yyyy-mm 형태로 변환
date_candidates = ['날짜', '기준년월', '기준일', 'date', '기준일자']
date_col = next((c for c in date_candidates if c in result_table.columns), None)
if date_col is not None:
    s = result_table[date_col].astype(str).str.strip()
    parsed = pd.to_datetime(s, errors='coerce', format='%Y%m')
    parsed = parsed.fillna(pd.to_datetime(s, errors='coerce', infer_datetime_format=True))
    result_table['날짜'] = parsed.dt.strftime('%Y-%m')
    if date_col != '날짜':
        result_table.drop(columns=[date_col], inplace=True)

# 시도 지역명 제외
exclude_regions = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '충북', '충남', '전남', '경북', '경남', '제주', '강원', '전북']
if '지역명' in result_table.columns:
    result_table = result_table[~result_table['지역명'].isin(exclude_regions)].reset_index(drop=True)

# 시도코드와 시도명 매핑 딕셔너리 생성
sido_map = {
    '11': '서울특별시', '26': '부산광역시', '27': '대구광역시', '28': '인천광역시', '29': '광주광역시', '30': '대전광역시', '31': '울산광역시',
    '36': '세종특별자치시', '41': '경기도', '43': '충청북도', '44': '충청남도', '46': '전라남도', '47': '경상북도', '48': '경상남도',
    '50': '제주특별자치도', '51': '강원특별자치도', '52': '전북특별자치도'
}

# '시도' 컬럼 생성: 지역코드 앞 두자리로 시도명 매칭
if '지역코드' in result_table.columns:
    result_table['시도'] = result_table['지역코드'].astype(str).str[:2].map(sido_map)

# '지역명' 컬럼명을 '시군구'로 변경
if '지역명' in result_table.columns:
    result_table.rename(columns={'지역명': '시군구'}, inplace=True)

# '지역코드' 컬럼명을 '시도코드'로 변경
if '지역코드' in result_table.columns:
    result_table.rename(columns={'지역코드': '시도코드'}, inplace=True)

# 컬럼 순서 지정
desired_order = ['시도', '시군구', '날짜', '전세가율', '시도코드', '매물종별구분']
final_cols = [c for c in desired_order if c in result_table.columns]
result_table = result_table[final_cols]

result_table

/tmp/ipython-input-2916207403.py:49: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = parsed.fillna(pd.to_datetime(s, errors='coerce', infer_datetime_format=True))


,시도,시군구,날짜,전세가율,시도코드,매물종별구분
0,서울특별시,종로구,2013-04,57.12,11,아파트
1,서울특별시,종로구,2013-05,57.13,11,아파트
2,서울특별시,종로구,2013-06,57.40,11,아파트
3,서울특별시,종로구,2013-07,57.82,11,아파트
4,서울특별시,종로구,2013-08,58.24,11,아파트
...,...,...,...,...,...,...
23380,전북특별자치도,부안군,2025-08,62.19,52,아파트
23381,전북특별자치도,부안군,2025-09,62.30,52,아파트
23382,전북특별자치도,부안군,2025-10,62.17,52,아파트
23383,전북특별자치도,부안군,2025-11,62.17,52,아파트
